# Juliet Tutorial

Juliet ist ein Programm, um die Transitparameter von Planeten zu finden basierend auf den Daten. Dazu benutzt es nested sampling. 

Mehr infos könnt ihr hier finden: https://juliet.readthedocs.io/en/latest/


## Zuerst laden wir die python packages, die wir benötigen:

In [ ]:
import juliet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astroquery.mast import Catalogs, Observations
from tess_stars2px import tess_stars2px_function_entry
import os
from astropy.io import fits
import corner
import astropy.constants as astrocon

## Daten
Wir haben bereits gelernt, wie man mit `lightkurve` Daten herunterlädt. Dies kann man auch durch mit astroquery machen. Dies speichert die Daten in einem Ordner, den man einfach angeben kann: 

Für das folgende Script nennt ihr diesen Ordner TESS und er sollte in dem Pfad sein, den ihr später angebt. Ansonsten müsst ihr ein paar Zeilen im Code ändern.


In [ ]:
tic=55092869

# Get SPOC light curves
obsTable = Observations.query_criteria(provenance_name="TESS-SPOC",
                                    target_name=tic,
                                    dataproduct_type='timeseries'
                                    )
    
data = Observations.get_product_list(obsTable)
print(data) # Check that this is not empty (else you might only have QLP data -> talk to me!)
download_lc = Observations.download_products(data,  extension="_lc.fits", download_dir='/Users/yoshi/Desktop/KELT11/')

Am einfachsten (oder so mache zumindest ich es) ist es, die Dateien in einen Ordner zu bewegen, den ihr leichter finden könnt und der nicht so verschachtelt ist. Dabei könnt ihr die Dateien auch zu TIC_Sektor.fits umbennen. Das macht das öffnen zu dem wir gleich kommen leichter. 

Hier geben wir den Pfad zu dem Ordner an, in dem die Dateien gespeichert sind:

In [ ]:
path='/Users/yoshi/Documents/Outreach/NAka25/'

Jetzt schauen wir, in welchen Sektoren der Stern observiert wurde. Das machen wir mit `tess-point`. Diese Funktion benötigt neben der TIC ID, die Koordinaten. Diese erhalten wir aus dem TIC.

Gib hierzu die TIC ID, die du auf ExoFOP gefunden hast, deines Sternes ein. 

In [ ]:
tic=55092869 #Enter the TIC ID here
# Query the TIC for that target
catalog_data = Catalogs.query_object(f"TIC {tic}", catalog="TIC")
row = catalog_data[0]
# Get the coordinates of the target from the TIC
ra = row['ra']
dec = row['dec']
# Use tess-point to obtain the sectors
result=tess_stars2px_function_entry(tic, ra, dec)
sectors=result[3]
#Filter since TESS is currently in sector 87 (Jan 2025) and a lot of more recent sectors aren't available yet
sectors=sectors[sectors<85]
print(sectors)

Um den Planeten Radius zu bestimmen, benötigen wir den Radius des Sterns. Auch dieser ist im TIC und wir rufen ihn hier schon mal auf für später.
Auch kann man von Transits die Dichte des Sterns bestimmen. Diese können wir mit den bekannten Werten für Radius und Masse ausrechnen:

In [ ]:
r_star=row['rad']*astrocon.R_sun.value
m_star=row['mass']*astrocon.M_sun.value # nan
print(r_star)
rhostar = m_star/((4*np.pi/3)*r_star**3) # Not working for KELT 11 since mass in nan in TIC (would have to enter a value manually)
print(rhostar)

Zuerst schauen wir uns die Daten von einem Sektor (in diesem Fall Sektor 9) an, sodass wir Transits sehen können:

In [ ]:
time=[]
flux=[]
# Select the sector
sec=9
# Open the fits file and access the data
lc=os.path.join(path,f"TESS/{tic}_{sec}.fits")
hdul = fits.open(lc)
t=hdul[1].data['TIME']
f=hdul[1].data['PDCSAP_FLUX'] # detrended flux 
# Removing bad quality data points
quality=hdul[1].data['QUALITY']
t=t[quality==0]
f=f[quality==0]
# Removing nan values
t=t[~np.isnan(f)]
f=f[~np.isnan(f)]
f=f/np.mean(f) # normalising the data
# appending the data to the lists above
time.extend(t)
flux.extend(f)

# Plotting
fig, ax = plt.subplots(figsize=(16,6))
ax.plot(time, flux, '.')
ax.set_xlabel('Time (TBJD)')
ax.set_ylabel('Detrended and normalised flux')
ax.set_title('KELT-11 TESS data, sector 9')
ax.minorticks_on()
plt.show()

Wir können nun probieren, den Transit zu modellieren, indem wir batman nutzen. In diesem Code plotten wir die Daten und das batman model. Spiel mit P, t0 und rp herum und schaue, ob Du es schaffst die Daten gut zu modellieren.

In [ ]:
# batman model
import batman

#Enter your guesses here
per= 5  
t0= 1549       
rp=0.04

params_kelt11b = batman.TransitParams()

params_kelt11b.t0 = t0
params_kelt11b.per = per
params_kelt11b.rp = rp
params_kelt11b.inc = 85.3
params_kelt11b.ecc = 0.00070
params_kelt11b.a = 4.98
params_kelt11b.w = 90.
params_kelt11b.limb_dark = "quadratic"
params_kelt11b.u = [0.5, 0.5]

# Set time array to the time of our data, so that we can see the model transits in the plot
batman_time = np.array(time)

batman_model = batman.TransitModel(params_kelt11b, batman_time)

batman_flux = batman_model.light_curve(params_kelt11b)


# Plot
fig, ax = plt.subplots(figsize=(16,6))
ax.plot(time, flux, '.', label='Data')
ax.plot(batman_time, batman_flux, c='r', label='Model')
ax.set_xlabel('Time [TBJD]')
ax.set_ylabel('Detrended and normalised flux')
ax.set_title('KELT-11 TESS data, sector 9')
ax.legend()
ax.minorticks_on()
plt.show()

Als nächstes lesen wir alles verfügbaren TESS-Daten ein. Dies könnt ihr in einem for-loop machen. Probiert es zunächst selbst!

In [ ]:
time=[]
flux=[]
flux_err=[]
# Looping over all observed sectors
for sec in sectors:
    try:
        lc=os.path.join(path,f"TESS/{tic}_{sec}.fits")
        hdul = fits.open(lc)
        t=hdul[1].data['TIME']
        f=hdul[1].data['PDCSAP_FLUX']
        ferr=hdul[1].data['PDCSAP_FLUX_ERR']
        quality=hdul[1].data['QUALITY']
        t=t[quality==0]
        f=f[quality==0]
        ferr=ferr[quality==0]
        t=t[~np.isnan(f)]
        ferr=ferr[~np.isnan(f)]
        f=f[~np.isnan(f)]
        ferr=ferr/np.mean(f)
        f=f/np.mean(f)
        time.extend(t+2457000) # Converting to BJD
        flux.extend(f)
        flux_err.extend(ferr)
    # in case any sectors are missing
    except FileNotFoundError:
        continue


times, fluxes, fluxes_error = {},{},{}
times['TESS'], fluxes['TESS'], fluxes_error['TESS'] = np.array(time),np.array(flux),np.array(flux_err)
print(times) # check that this is not empty

Jetzt können wir alle Daten plotten. In diesem Fall haben wir 2 Sektoren.

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
ax.plot(time, flux, '.')
ax.set_xlabel('Time [BJD]')
ax.set_ylabel('Detrended and normalised flux')
ax.set_title('KELT-11 TESS data, sectors 9 and 62')
ax.minorticks_on()

## Vorbereitung des Fits

Um die Transits zu modellieren, fitten wir Daten. Das Model benötigt dazu inputs, damit es ein Model zum Anfangen hat. Es wird die Werte, die wir ihm geben, optimieren.
Dazu geben wir P und t0 hier ein (wir ihr sie auf ExoFOP oder Exoplanet Archive findet).

Die anderen Parameter können wir in Ruhe lassen (vertraut mir).

In [ ]:
P=4.7361 #in days
t0=2457483.431 #in BJD

In [ ]:
priors = {}

# Name of the parameters to be fit:
params = ['P_p1','t0_p1','p_p1','b_p1','q1_TESS','q2_TESS','ecc_p1','omega_p1',\
              'rho', 'mdilution_TESS', 'mflux_TESS', 'sigma_w_TESS']

# Distributions:
dists = ['normal','normal','uniform','uniform','uniform','uniform','fixed','fixed',\
                 'loguniform', 'fixed', 'normal', 'loguniform']

# Hyperparameters
hyperps = [[P,0.1], [t0,0.1], [0.,1], [0.,1.], [0., 1.], [0., 1.], 0.0, 90.,\
                   [100., 10000.], 1.0, [0.,0.1], [0.1, 1000.]]

# Populate the priors dictionary:
for param, dist, hyperp in zip(params, dists, hyperps):
    priors[param] = {}
    priors[param]['distribution'], priors[param]['hyperparameters'] = dist, hyperp

Hier geben wir ein, wie der Ordner heißen soll, in dem wir das Ergebnis speichern:

In [ ]:
outdir=path+'KELT11_TESS/'

## Jetzt lassen wir juliet seine Arbeit machen 

In [ ]:
dataset = juliet.load(priors=priors, t_lc = times, y_lc = fluxes, 
                   yerr_lc = fluxes_error, out_folder = outdir)

results = dataset.fit(n_live_points=300)

## Ergebnisse

Nach einiger Zeit sollte die Zelle ausgeführt sein und wir können uns die Ergebnisse anschauen:

In [ ]:
# Get the model
transit_model, transit_up68, transit_low68 = results.lc.evaluate('TESS', return_err=True)

# Get P and t0
P, t0 = np.median(results.posteriors['posterior_samples']['P_p1']),\
        np.median(results.posteriors['posterior_samples']['t0_p1'])

# Get the times, flux and model
t=dataset.times_lc['TESS']
f=dataset.data_lc['TESS']
ferr=dataset.errors_lc['TESS']
model=transit_model
# Get the phase
phase=juliet.get_phases(t, P, t0)
# Plot all
plt.errorbar(phase, f,yerr=ferr, fmt = '.', alpha=0.1)
idx = np.argsort(phase)
p_bin, y_bin, yerr_bin = juliet.bin_data(phase[idx], f[idx], 200)
plt.errorbar(p_bin, y_bin, yerr = yerr_bin, fmt = 'o', mfc = 'white', mec = 'black', ecolor = 'black')

plt.plot(phase[idx], model[idx], c='black', zorder=11)
plt.fill_between(phase[idx],(transit_up68)[idx],(transit_low68)[idx],color='white',alpha=0.5,zorder=5)
plt.xlim(-0.2, 0.2)
plt.ylabel('Relative Flux')
plt.xlabel('Phases')
plt.show()


Wir lassen uns noch ein paar praktische Werte ausgeben:

In [ ]:
Pm1,Pu1,Pl1 = juliet.utils.get_quantiles(results.posteriors['posterior_samples']['P_p1'])
t0m1,t0u1,t0l1 = juliet.utils.get_quantiles(results.posteriors['posterior_samples']['t0_p1'])
pm1,pu1,pl1 = juliet.utils.get_quantiles(results.posteriors['posterior_samples']['p_p1'])

In [ ]:
print(Pm1, Pm1-Pu1, Pl1-Pm1)

In [ ]:
print(t0m1, t0m1-t0u1, t0l1-t0m1)

In [ ]:
print(pm1, pm1-pu1, pl1-pm1)